To do:

- chemische Reaktion für Repressor richtig darstellen und so einfügen das k+ und k- angezeigt werden
-  autoregulation repressor übnerarbeiten

In [ ]:
// Import the required libraries
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: FsODE, 0.0.1"
#r "nuget: FSharp.Stats"

// ... and open the required modules
open FsODE
open Plotly.NET


let modelContext = //OdeContext()
    OdeSolverMethod.RK546M //RK547M()
    |> OdeContext

Installed Packages FSharp.Stats, 0.4.7 FsODE, 0.0.1 Plotly.NET.Interactive, 3.0.2

## Tag 3 - Modellierung von biologischen Systemen

### Inhalt

### Biologische Systeme 

Biologische Systeme können als Netzwerke aufgefasst werden, in welchen biologischen Einheiten miteinander verknüpft sind. Dabei hängen Größe, Art, und Anzahl der biologischen Einheiten vom gewählten System ab. Biologische Zellen sind dabei besonders interessanteste Systeme. Sie werden begrenzt von einer Membran die Stoff, Energie, und Informationsaustausch mit der Umgebung erlaubt, und enthalten Millionen von Molekülen die alle miteinander interagieren. Dabei sind die Interaktionen zumeist spezifischer Natur und formen selbst wiederum kleinere biologische Systeme, die die Funktion der ganzen Zelle sicherstellen. Beispiele für solche Systeme sind die Kontrolle des Zelltods, Signalwege die die Kommunikation mit anderen Zellen erlauben, die Nutzung von Metaboliten, und viele weitere. 
Die Grundeinheiten in solchen Systemen sind Gene, beziehungsweise die Proteine, welche von diesen kodiert werden. Daher ist es notwendig herauszufinden, wie hoch die Konzentration der einzelnen Proteine zu einem bestimmten Zeitpunkt ist und wie diese sich über die Zeit verändert. Dieses Verhalten kann allgemein durch ODEs beschrieben werden, wobei der Aufbau der Differenzialgleichung immer identisch ist und sich nur die Komplexität von dieser ändert, in Abhängigkeit der Anzahl an Faktoren, die auf die Produktion des Proteins Einfluss nehmen. Diese allgemeine Form ist dabei
$$ \frac {dProteinkonzentration} {dt} = \textrm{Produktion des Proteins} - \textrm{Verringerung der Proteinkonzentration}$$

Was ist dann die Proteinkonzentration eines Proteins X das von einem Gen x produziert wird? Wenn die Annahme getätigt wird, dass das Gen x nur das Protein X codiert und auch in sonst keiner Weise beeinflusst wird, kann die Produktion des Proteins X direkt mit einer Produktionsrate $\beta$ beschrieben werden. Die Verringerung der Proteinkonzentration von X ist auf der einen Seite abhängig vom aktiven Abbau in der Zelle, und auf der anderen von der Verteilung des Proteins in der Zelle während diese wächst und sich teilt. Der einfachheitshalber, können diese Faktoren allerdings als ein Prozess betrachtet werden. Die mathematisch Beschreibung dieses Prozesses ist dann eine Verringerung der Proteinkonzentration proportional zu einem Verringerungsfaktor $\gamma$. Die Differenzialgleichung ist somit

$$ \frac {dProteinkonzentration} {dt} = \beta - \gamma*Proteinkonznetration$$

Aufgabe:
Setze die Differenzialgleichung in das folgende Model ein (beachte das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole).


In [ ]:
//Konstanten
let beta =    1. // Konstante für die Zunahme der Proteinkonzentration
let gamma =   5. // Konstante für die Verringerung der Proteinkonzentration

// Definiere eine Funktion welche die Proteinkonzentration in Abhägngikeit von der Zeit beschreibt
let dP_dt : Model = 
    fun P t ->
        // Veränderung der Proteinmenge
        let Proteinkonznetration = P[0] 
        // Modelfunktion
        let Proteinkonznetration' = beta - gamma*Proteinkonznetration

        [| Proteinkonznetration' |]
// Anfangsbedingungen
let P0 = [| 
    10.0 
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        1., // start time point
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // n timepoints
    |> SolPoints.memorize 

let proteins = Ps |> SolPoints.toPoints 1

//Graphische Darstellung der numerischen Lösung
proteins
|> Chart.Spline 
|> Chart.withTraceInfo("Protein")
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Aufgabe: 
Der gezeigte Graph zeigt die Proteinkonzentration in Abhängigkeit von der Zeit. Mit den aktuell gewählten Werten für die beiden Konstanten $\beta \textrm{ und } \gamma$ befindet sich der Gleichgewichtszustand (GGW) für die Proteinkonzentration bei einem Wert nahe Null. Verändere $\beta$ und versuche eine Gesetzmäßigkeit zu erkennen die beschreibt welchen Wert die Proteinkonzentration im GGW annimmt in Abhängigkeit von $\beta \textrm{ und } \gamma$. Wähle die korrekte Gleichung aus den vier nachfolgen aus, die diese Gesetzmäßigkeit beschreibt:


a) $Proteinkonzentration_{GGW} = \gamma  - \beta$
 
b) $Proteinkonzentration_{GGW} = \frac {\beta+ \gamma}  {\gamma^{2}}$ 

c) $Proteinkonzentration_{GGW} = \frac {\beta}  {\gamma}$ 

d) $Proteinkonzentration_{GGW} = \frac {\beta^{2}}  {\gamma^{2}}$ 

### Aktivator und Repressor 

Leider ist die die Biologie in fast allen fällen nicht so einfach. Nahezu alle Gene sind nicht dauerhaft aktiv und produzieren mRNA, die dann translatiert wird in Proteine. Stattdessen werden Gene in der Regel reguliert, um die Genexpression an die Umgebungsbedingungen anzupassen. Die einfachste Form der Regulierung ist mittels Repressoren und Aktivatoren. Beides sind Proteine, die an spezifische Regionen auf der DNA binden können um die Promotoraktivität eines Gens zu regulieren, wobei die Stärke der Regulierung meist mit der Stärke der Bindung des Regulators zur DNA-Region einhergeht. 

Überleitung einfügen!!

Innerhalb der Zelle bindet und löst sich ein Repressor von seiner Bindungsstelle in einem kontinuierlichen Prozess. Dies kann als eine Gleichgewichtsreaktion dargestellt werden
$$D + R <=> D_{gebunden}$$
wobei $D$ die Konzentration von Bindungsstellen ohne Repressor, $R$ die Repressorkonzentration, $D_{gebunden}$ die Konzentration von Bindungsstellen mit Repressor, $k_{+}$ die Geschwindigkeit des Bindens des Repressor an die Bindungstelle, und $k_{-}$ die Geschwindigkeit des Loslösens des Repressors von der Bindungstelle ist. $D$ und $D_{gebunden}$ zusammen ergeben $D_{total}$ was der Konzentration aller Bindungstellen entspricht ($D_{total} = D + D_{gebunden} $).
Das Gen wird nur dann stark exprimiert wenn der Repressor nicht gebunden ist, was dem Verhältnis $D/D_{total}$ entspricht. Für eine Differenzialgleichung die beschreibt wie sich die Proteinkonzentration verhält wenn das Gen von einem Repressor reguliert wird ist es nötig die Produktionsrate mit dem Verhältnis $D/D_{total}$ zu multipliziere. Da dies beschreibt, wie häufig das Gen ohne Repressor anzufinden ist. Sowohl $D$ als auch $D_{total}$ sind aber nur schwierig experimentell zu bestimmen. Die Repressorkonzentration und die beiden Geschwindigkeiten sind hingegen vergleichsweise einfach zu bestimmen. Daher ist es sinnvoll das Verhältnis $D/D_{total}$ als einen Zusammenhang von $R>$ und den Geschwindigkeiten darzustellen. Dies ist möglich durch das Umformen der Gleichgewichtsgleichung
$$k_{+}*D*R = k_{-}*D_{gebunden}$$
Einsetzten von $D_{total} = D + D_{gebunden} $ um $D_ {gebunden}$ zu ersetzen ergibt 
$$k_{+}*D*R = k_{-}*(D_{total}-D)$$
Seperieren von $D$ und $D_{total}$ von den anderen Faktoren führt zu
$$\frac {D} {(D_{total}-D)}= \frac {k_{-}}{k_{+}*R} $$
$$\frac {D/D} {(D_{total}/D-D/D)}= \frac {k_{-}}{k_{+}*R} $$
$$\frac {1} {(D_{total}/D-1)}= \frac {k_{-}}{k_{+}*R} $$
$$\frac {D_{total}}{D}-1= \frac {k_{+}*R}{k_{-}} $$
$$\frac {D_{total}}{D}= \frac {k_{+}*R}{k_{-}}+1 $$
die Vereinfachung $K_{D} = \frac {k_{-}}{k_{+}}$ führt schließlich zu
$$\frac {D_{total}}{D}= \frac {R}{K_D} +1 $$
$$\frac {D}{D_{total}}= \frac {1}{1+\frac {R}{K_{D}}} $$

Der Term $\frac {1}{1+\frac {R}{K_{D}}}$ beschreibt somit den Term durch den die Expressionsrate eines Gens gedämpft wird, wenn es von einem Repressor inhibiert wird. Das heißt wenn das Gen x, welches das Protein X produziert, durch den Inhibitor R reguliert wird kann dies mit folgender Differenzialgleichung beschrieben werden
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {1}{1+ \frac {R}{K_{D}}}  - \gamma*Proteinkonznetration$$
wobei $K_{D}$ die Konzentration ist, bei der die Produktion um die Hälfte reduziert ist. 





Aufgabe:<br>
Setze die Differenzialgleichungen für ein Protein dessen Produktion von einem Repressor inhibiert wird in das folgende Model ein (beachte das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{d}$ als K_d eingesetzt wird).


In [ ]:
//Konstanten
let alpha =       0.5  // 
let beta =    50.  // Konstante für die Zunahme der Proteinkonzentration
let gamma =   20.  // Konstante für die Verringerung der Proteinkonzentration
let K_d =     2.   // Repressorkonzentration bei der die Proteinproduktion um die Hälfte reduziert ist
let R =       10.  // Repressorkonzentration


// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->

        let x = P[0] // Change in protein quantity
        let y = P[1]
        //Änderung der Proteinkonzentration ohne einen Repressor
        let x' = beta - (gamma*x)
        // Änderung des Proteinkonzentration unter Einfluß von einem Repressor
        let y' = beta * 1./(1.+(R/K_d)) - (gamma*y)

        [| x'; y' |]

let P0 = [| 
    0.0
    0.0  // x(0), protein quantity at start (x(t) -> x(0))
|]

let Ps = 
    modelContext.OdeInt(
        1., // start time point
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // n timepoints
    |> SolPoints.memorize 
[
    let proteins = Ps |> SolPoints.toPoints 1 
    proteins |> Chart.Spline 
    |> Chart.withTraceInfo("Unreguliert");
    let proteinsregulated = Ps |> SolPoints.toPoints 2 
    proteinsregulated|> Chart.Spline 
    |> Chart.withTraceInfo("Reguliert");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Aufgabe: <br>
Erklären sie wie sich der Repressor auf die Höhe des Gleichgewichtszustandes der Proteinkonzentration und der Zeit die benötigt wird diesen zu erreichen auswirkt. Verändern sie dafür die Werte von "R" falls nötig. Beachten sie das die eingesetzten Werte reale Zahlen sein müssen und nicht ganze Zahlen.


Aufgabe: <br>
In der Realität wird ein Gen schwach exprimiert, auch wenn die Expression des Gens durch einen Repressor gehemmt wird. Ergänze die Differenzialgleichungen für ein Protein dessen Produktion von einem Repressor inhibiert wird um einen Term $\alpha$ der diese Grundgenexpression wiederspielt. Beachte dabei das $\alpha$ nicht vom Repressor beeinflusst wird (setze $\alpha$ als "alpha", ohne "", in die Gleichung ein).<br>
Wie wirkt sich $\alpha$ auf den Gleichgewichtszustand auf?


Aufgabe:<br>
Stelle die Differenzialgleichung für die Proteinproduktion auf, die von einem Aktivator reguliert wird. Beachte dabei das ein Aktivator gegensätzlich zu einem Repressor wirkt und daher der Faktor mit dem $beta$ multipliziert werden muss $\frac {D_{gebunden}}{D_{total}}$ ist. Dies bedeutet das sie folgende Gleichung in diese Form umwandeln müssen
$$k_{+}*(D_{gebunden}-D_{total})*R = k_{-}*D_{gebunden}$$
und anschließend in die Differenzialgleichung für die Proteinproduktion einsetzen müssen
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {D_{gebunden}}{D_{total}}  - \gamma*Proteinkonznetration$$ 

Falls sie Probleme mit dem Umformen haben, schauen sie sich noch einmal die Umformung für den Repressor an.

Setze die Differenzialgleichungen für ein Protein das von einem Aktivator reguliert wird in das folgende Model ein.


In [ ]:


//Konstanten
let beta =    10.  // Konstante für die Zunahme der Proteinkonzentration
let gamma =   5.  // Konstante für die Verringerung der Proteinkonzentration
let K_d =     2.   // Repressorkonzentration bei der die Proteinproduktion um die Hälfte reduziert ist
let R =       10.  // Repressorkonzentration
let A =       10.  // Aktivatorexpression 

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->

        let x = P[0] // Change in protein quantity
        let y = P[1]
        let z = P[2]
        //Änderung der Proteinkonzentration ohne einen Repressor
        let x' = beta - (gamma*x)
        // Änderung des Proteinkonzentration unter Einfluß von einem Repressor
        let y' = beta * 1./(1.+(R/K_d)) - (gamma*y)
        // Änderung des Proteinkonzentration unter Einfluß von einem Aktivator
        let z' = beta * 1./(1.+(K_d/A)) - (gamma*z)

        [| x'; y';z' |]
//Anfangsbedingungen (x0, y0, z0)
let P0 = [| 
    0.0
    0.0
    0.0  
|]

let Ps = 
    modelContext.OdeInt(
        1., // start time point
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // n timepoints
    |> SolPoints.memorize 
[
    let proteins = Ps |> SolPoints.toPoints 1 
    proteins |> Chart.Spline 
    |> Chart.withTraceInfo("Unreguliert");
    let proteinsregulatedbyrep = Ps |> SolPoints.toPoints 2 
    proteinsregulatedbyrep|> Chart.Spline 
    |> Chart.withTraceInfo("Repressor");
    let proteinsregulatedbyakt = Ps |> SolPoints.toPoints 3 
    proteinsregulatedbyakt|> Chart.Spline 
    |> Chart.withTraceInfo("Aktivator");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Es folgt eine kurze Besprechung. Falls Sie schnell vorangekommen sind, können sie bis zur Besprechung folgende Aufgaben lösen:

Expertenaufgaben <br>
Aktivator und Repressor führen beide zu einer Verringerung der maximalen Genexpression. Wenn allerdings beide zum selben Ergebnis führen, warum gibt es dann beides? Erklären sie, warum mache Gene eher von einem Aktivator reguliert werden sollten als von einem Repressor und umgekehrt.

### Autoregulation

Neben der Fähigkeit von Repressoren die Produktion von bestimmten Proteinen zu regulieren, sind viele auch dazu in der Lage sich selbst zu regulieren. Ein Beispiel dafür ist der Repressor LacI, der nicht nur die Expression des Lac Operons begrenzt sondern auch seine eigene. Dies wird negative Autoregulation genannt. Die Differenzialgleichung, die diesen Prozess beschreibt ist analog zu die von der Proteinkonzentration die von einem Repressor reguliert wird, nur das die Repressorkonzentration identisch ist mit der Proteinkonzentration, also
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {1}{1+ \frac {Proteinkonznetration}{K_{D}}}  - \gamma*Proteinkonznetration$$


